In [ ]:
from google.colab import files
import cv2
import numpy as np
import ipywidgets as widgets
from PIL import Image, ImageDraw, ImageFont

# 画像のアップロードフォームを表示
uploaded = files.upload()

# アップロードされた画像のファイル名を取得
image_filename = next(iter(uploaded))

# アップロードされた画像を読み込む
image = cv2.imdecode(np.frombuffer(uploaded[image_filename], np.uint8), -1)

# 画像をグレースケールに変換
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 使いたい文字を入力するフォームを表示
char_input = widgets.Text(
    value = '',
    placeholder = 'Enter characters',
    description = 'Characters:',
    disabled = False
)
display(char_input)

# 濃淡に応じた文字列を取得する関数
def intensity_to_char(intensity, chars):
    index = int(intensity / 255 * (len(chars) - 1))
    return chars[index]

def generate_ascii_image(gray_image, chars):
    # イラスト化した文字画像を生成
    ascii_image = ""
    for i in range(gray_image.shape[0]):
        for j in range(gray_image.shape[1]):
            intensity = gray_image[i, j]
            ascii_image += intensity_to_char(intensity, chars)
        ascii_image += "\n"
    return ascii_image

def on_button_clicked(b):
    chars = char_input.value

    # 文字列が空でないことを確認
    if chars:
        # グレースケール画像を元にイラスト化した文字画像を生成
        ascii_image_text = generate_ascii_image(gray_image, chars)

        # 生成されたテキストデータから画像を作成
        font = ImageFont.load_default()
        text_width, text_height = font.getsize("X")
        img_width = gray_image.shape[1] * text_width
        img_height = gray_image.shape[0] * text_height
        img = Image.new("RGB", (img_width, img_height), color=(255, 255, 255))
        draw = ImageDraw.Draw(img)
        draw.text((0, 0), ascii_image_text, fill=(0, 0, 0), font=font)

        # 画像を表示
        display(img)
    else:
        print("Please enter characters.")

# 決定ボタンの作成
button = widgets.Button(description="決定")
button.on_click(on_button_clicked)
display(button)